In [1]:
import numpy as np
import tensorflow as tf 
from collections import deque
import random
import pdb
from replay_memory_Q import ReplayMemory
from doubleq_networks import DoubleQ_Network
from test_game import test_env
from info_obj import info_holder
import os
from __future__ import print_function

In [2]:
import gym
game_name = 'LunarLander-v2'
#game_name = 'CartPole-v0'
#game_name = 'MountainCar-v0'
game = gym.make(game_name)
state_dim = game.observation_space.shape[0]
action_dim = game.action_space.n
'''
game = test_env()
state_dim = 1
action_dim = 2
'''

[2017-01-02 17:25:09,769] Making new env: LunarLander-v2


'\ngame = test_env()\nstate_dim = 1\naction_dim = 2\n'

In [3]:
# info = graph, dim_s, dim_a, hid_layers, learning_rate, tau
if_load = 'n'
hid_lay = [64, 64]
save_dir = './Results/'+ game_name[:-3] + '/net/'
if 'LunarLander' in game_name:
    BUFFER_SIZE = int(1e5)
    MINIBATCH_SIZE = 128
    LEARNING_RATE = 1e-3 #5e-5 lr/2
    lr_decay_rate = 1
    EPSILON = 1
    eps_decay_rate = 0.9
    C_target_update = 16 #episode
    GAMMA = 0.99
    MAX_EPISODE = int(3e4)
    MAX_STEP = int(1e3)
    lr_decay_ep = 100
    print_ep = int(1e2)
    save_ep = int(1e3)
    TAU = 0
    ENTROPY_BETA = 0
    
elif 'CartPole' in game_name:
    BUFFER_SIZE = int(1e5)
    MINIBATCH_SIZE = 32
    LEARNING_RATE = 1e-4 #5e-5 lr/2
    lr_decay_rate = 1
    EPSILON = 1
    eps_decay_rate = 0.955
    C_target_update = 8 #episode
    GAMMA = 0.99
    MAX_EPISODE = int(1e4)
    MAX_STEP = int(2e2)
    lr_decay_ep = 100
    print_ep = int(1e2)
    save_ep = int(1e3)
    TAU = 0
    ENTROPY_BETA = 0
    
elif 'MountainCar' in game_name:
    BUFFER_SIZE = int(1e5)
    MINIBATCH_SIZE = 256
    LEARNING_RATE = 1e-2 #5e-5 lr/2
    lr_decay_rate = 0.955
    EPSILON = 1
    eps_decay_rate = 0.955
    C_target_update = 100 #episode
    GAMMA = 0.99
    MAX_EPISODE = int(1e4)
    MAX_STEP = int(2e2)
    lr_decay_ep = 100
    print_ep = int(1e2)
    save_ep = int(1e3)
    TAU = 0
    ENTROPY_BETA = 0

MIN_EPSILON = 0.1
#### load the network
logging_mode = 'w'
strings = ['Learning rate = ', 'lr decay = ', 'Epsilon = ', 'eps decay = ', 'Buffer size = ', 'Batch size = ',\
           'hidden layers: ', 'target update cycle: ']
if 'y' == if_load:
    if not(os.path.isdir(save_dir)):
        raise NameError('No network to upload')
    logging_mode = 'a'
    with open(save_dir + 'params.txt', 'r') as f:
        xa = f.readline()
        while xa:
            if strings[0] in xa:
                LEARNING_RATE = float(xa[len(strings[0]):])
            if strings[1] in xa:
                lr_decay_rate = float(xa[len(strings[1]):])
            elif strings[2] in xa:
                EPSILON = float(xa[len(strings[2]):])
            elif strings[3] in xa:
                eps_decay_rate = float(xa[len(strings[3]):])
            elif strings[4] in xa:
                BUFFER_SIZE = int(xa[len(strings[4]):])
            elif strings[5] in xa:
                MINIBATCH_SIZE = int(xa[len(strings[5]):])
            elif strings[6] in xa:
                hid_lay = [int(xb) for xb in xa[len(strings[6]):].split()]
            elif strings[7] in xa:
                C_target_update = int(xa[len(strings[7]):])
            xa = f.readline()
    with open(save_dir + 'rewards.txt', 'r') as f:
        xa = f.readline()
        xa = xa.split(',')
        start_ep = int(xa[0]) + 1
    var = raw_input("From what episode?(Enter a number or last): ")
    if var != 'last': 
        var_int = int(var)
        while ((var_int%save_ep)!=0) or (var_int > start_ep):
            print('Episode not found!')
            var = raw_input("From what episode?(Enter a number or last): ")
    print('Loading the previous trained network and continue training from episode %d' %start_ep)
else:
    start_ep = 0
lr = LEARNING_RATE * (lr_decay_rate ** (start_ep/lr_decay_ep))# / MINIBATCH_SIZE
eps = EPSILON * (eps_decay_rate ** (start_ep/lr_decay_ep))
if eps < MIN_EPSILON:
    eps = MIN_EPSILON
#### till here

    
buff = ReplayMemory(BUFFER_SIZE)
info = info_holder()
try:
    DQN
except:
    pass
else:
    DQN.reset_net()

graph = tf.Graph()
with graph.as_default():
    info.graph = graph
    info.dim_s = state_dim
    info.dim_a = action_dim
    info.hid_layers = np.array(hid_lay)
    info.learning_rate = LEARNING_RATE
    info.tau = TAU
    info.entropy_beta = ENTROPY_BETA
    DQN = DoubleQ_Network(info)

In [4]:
#eps = EPSILON
#lr = LEARNING_RATE
buff.clear()
last_ep = 0
start_update = 0
with tf.Session(graph=graph) as sess:
    DQN.set_up(sess)
    sess.run(tf.global_variables_initializer())
    
    if if_load == 'y':
        saver = tf.train.Saver()
        filename = save_dir + 'Episod_' + str(start_ep-1) + '.chk'
        saver.restore(sess, filename)
        buff.load_buf(save_dir)
    #pdb.set_trace()
    ep_reward = np.zeros(MAX_EPISODE)
    ep_success = np.zeros(MAX_EPISODE)
    ep_crash = np.zeros(MAX_EPISODE)
    batch_loss = np.zeros(MAX_EPISODE)
    #avg_var = 0
    
    for ep in range(MAX_EPISODE):
        s = game.reset()
        #var = 0
        for st in range(MAX_STEP):
            a = DQN.take_action(s.reshape(-1,info.dim_s), eps)
            s2, r, terminal, report = game.step(a)
            
            buff.add(np.reshape(s, (info.dim_s,)).astype(np.float32), np.array(a).astype(np.float32),\
                     np.array(r).astype(np.float32), np.reshape(s2, (info.dim_s,)).astype(np.float32),\
                     np.array(terminal).astype(np.float32))
            
            ep_reward[ep] += (1.*r)
            #var += np.sum(actor.give_policy(s.reshape(-1,info.dim_s))**2)
            s = s2
            if buff.size() >= MINIBATCH_SIZE:
                (s_batch, a_batch, r_batch, s2_batch, t_batch) = buff.sample_batch(MINIBATCH_SIZE)
                
                s_batch = np.array(s_batch).reshape(-1, info.dim_s)
                s2_batch = np.array(s2_batch).reshape(-1, info.dim_s)
                a_batch = np.array(a_batch)#.reshape(-1, 1)
                r_batch = np.array(r_batch)#.reshape(-1, 1)
                t_batch = np.array(t_batch)#.reshape(-1, 1)

                Q_max = DQN.target_q(s2_batch)
                Q_max = Q_max[0]
                
                R_batch = r_batch + GAMMA * Q_max * (1-t_batch) # if terminal then the second part is zero

                if(len(a_batch.shape)>1):
                    pdb.set_trace()
                    print(a_batch.shape, buff.size(), ep, st)
                batch_loss[ep], _ = DQN.train(s_batch, a_batch, R_batch, lr)
            
            if terminal:
                break
                
        if ((ep % C_target_update) == (C_target_update -1)):
            DQN.update_target()
                
        if (r >= 100):
            ep_success[ep] = 1
        elif (r <= -100):
            ep_crash[ep] = 1
        
        if ((ep % print_ep) == (print_ep-1)) or (ep == 0):
            print('Ep: %d --- Ave R: %.1f  |  loss: %.1f  |  Suc: %.2f  |  crash: %.2f  |  Max R: %.1f  |  lr:%.0fe-4  |  eps:%.0fe-2'\
                  %((start_ep+ep), np.mean(ep_reward[last_ep:ep+1]), np.mean(batch_loss[last_ep:ep+1]),\
                    np.mean(ep_success[last_ep:ep+1]), np.mean(ep_crash[last_ep:ep+1]), \
                    np.amax(ep_reward[last_ep:ep+1]), 1e4*lr, 1e2*eps))
            last_ep = ep
            
        if (ep == (save_ep-1)) and (if_load == 'n'):  ####
            os.mkdir(save_dir)  
            with open(save_dir + 'params.txt', 'w') as f:
                f.write('Learning rate = %f\n' %LEARNING_RATE)
                f.write('lr decay = %f\n' %lr_decay_rate)
                f.write('Epsilon = %f\n' %EPSILON)
                f.write('eps decay = %f\n' %eps_decay_rate)
                f.write('Buffer size = %d\n' %BUFFER_SIZE)
                f.write('Batch size = %d\n' %MINIBATCH_SIZE)
                f.write('hidden layers: %s\n' %str(info.hid_layers)[1:-1])
                f.write('target update cycle: %d\n' %C_target_update)
                
        if (ep == 0) and (if_load == 'n'):  ####
            if (os.path.isdir(save_dir)):
                raise NameError('Directory net exists')
                
        if (ep % save_ep) == (save_ep-1):
            print(' ***** Saving...  at episode %d ...' %(start_ep+ep), end="")
            saver = tf.train.Saver()
            filename = save_dir + 'Episod_' + str(start_ep+ep) + '.chk'
            saver.save(sess, filename)
            with open(save_dir + 'rewards.txt', logging_mode) as f:
                f.write('%d, , , , ,Learning rate = %f - lr decay = %f - Epsilon = %f - eps decay = %f - Buffer size = %d - Batch size = %d - hidden layers: %s - target update cycle: %d\n' \
                        %((start_ep+ep), LEARNING_RATE, lr_decay_rate, EPSILON, eps_decay_rate, BUFFER_SIZE, MINIBATCH_SIZE, str(info.hid_layers)[1:-1], C_target_update) )
                for i in range(ep):
                    f.write('%f,      %f\n' %(ep_reward[i], batch_loss[i]))
            buff.save_buf(save_dir)
            print(' Done ******')
        
            
        if (ep % lr_decay_ep) == (lr_decay_ep-1):
            lr = lr_decay_rate * lr
            eps = eps_decay_rate * eps
            if eps < MIN_EPSILON:
                eps = MIN_EPSILON
            
print('DONE') 
    

Ep: 0 --- Ave R: -367.9  |  loss: 0.0  |  Suc: 0.00  |  crash: 1.00  |  Max R: -367.9  |  lr:10e-4  |  eps:100e-2
Ep: 99 --- Ave R: -223.1  |  loss: 63.9  |  Suc: 0.00  |  crash: 1.00  |  Max R: -33.4  |  lr:10e-4  |  eps:100e-2
Ep: 199 --- Ave R: -201.9  |  loss: 31.4  |  Suc: 0.00  |  crash: 1.00  |  Max R: -39.3  |  lr:10e-4  |  eps:90e-2
Ep: 299 --- Ave R: -146.8  |  loss: 19.8  |  Suc: 0.00  |  crash: 0.99  |  Max R: 6.3  |  lr:10e-4  |  eps:81e-2
Ep: 399 --- Ave R: -128.1  |  loss: 19.2  |  Suc: 0.00  |  crash: 1.00  |  Max R: -29.8  |  lr:10e-4  |  eps:73e-2
Ep: 499 --- Ave R: -113.3  |  loss: 13.2  |  Suc: 0.00  |  crash: 1.00  |  Max R: -19.0  |  lr:10e-4  |  eps:66e-2
Ep: 599 --- Ave R: -105.6  |  loss: 12.4  |  Suc: 0.00  |  crash: 1.00  |  Max R: -37.0  |  lr:10e-4  |  eps:59e-2
Ep: 699 --- Ave R: -94.7  |  loss: 16.7  |  Suc: 0.00  |  crash: 0.96  |  Max R: 59.2  |  lr:10e-4  |  eps:53e-2
Ep: 799 --- Ave R: -97.9  |  loss: 13.9  |  Suc: 0.00  |  crash: 0.98  |  Max R: 63.6

KeyboardInterrupt: 

In [ ]:
EPSILON * (eps_decay_rate ** (start_ep/lr_decay_ep))